In [9]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')


In [8]:
def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2
    

In [3]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [4]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [17]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)
    

## 這裡開始要調參數

In [14]:
## 
n='Input_A1_020' # 改名稱
##
dire= ['../BayesianRidge_Pre_0/','../BayesianRidge_Pre_1/','../BayesianRidge_Pre_2/','../BayesianRidge_Pre_3/','../BayesianRidge_Pre_4/','../BayesianRidge_Pre_5/','../BayesianRidge_Pre_6/','../BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=numcorr['Predict'].sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))


Input_A1_020
../BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.5844720276614522
test_rmse: 0.6511332225562532
test_r2: 0.36485135531258117
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642275792
feature number: 5
train_rmse: 0.5824025653306407
test_rmse: 0.6594662460327358
test_r2: 0.3493447817400817
feature number: 6
train_rmse: 0.5820069840234381
test_rmse: 0.659608719896304
test_r2: 0.34914455883802875
feature number: 7
train_rmse: 0.5817708592190032
test_rmse: 0.6596553345534402
test_r2: 0.3489464324693693
feature number: 8
train_rmse: 0.5807490080212043
test_rmse: 0.6589953026259924
test_r2: 0.3504795277327843
feature number: 9
train_rmse: 0.5744447654257983
test_rmse: 0.651097957957286
test_r2: 0.3651818014507157
feature number: 10
train_rmse: 0.5669997827576497
test_rmse: 0.6465522376670974
test_r2: 0.37402471478653754


train_rmse: 0.5576502527833281
test_rmse: 0.6570853594050832
test_r2: 0.35146590597350413
feature number: 18
train_rmse: 0.5571385499118017
test_rmse: 0.6586092704498248
test_r2: 0.3482369390769565
feature number: 19
train_rmse: 0.5568632310975452
test_rmse: 0.6604571188227123
test_r2: 0.34471639327231884
feature number: 20
train_rmse: 0.5564922153592999
test_rmse: 0.6614624875390396
test_r2: 0.34278313997123894
Test_rmse_min = 1.fea_num: 13  2.rmse: 0.6460566545719896
Test_r2_max = 1. fea_num: 13  2.r2: 0.3744798876006497
../BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.5928217486273215
test_rmse: 0.646585647097155
test_r2: 0.370698679108309
feature number: 3
train_rmse: 0.5844720276614522
test_rmse: 0.6511332225562532
test_r2: 0.36485135531258117
feature number: 4
train_rmse: 0.5831209556536728
test_rmse: 0.6568360159671985
test_r2: 0.3545339642275792
feature number: 5
train_rmse: 0.5824025653306407
test_rmse: 0.6594662460327358
test_r2: 0.3493447817400817
feature number: 6
tr

train_rmse: 0.561332455655257
test_rmse: 0.6461008322537474
test_r2: 0.37433818993363543
feature number: 14
train_rmse: 0.5594664133854501
test_rmse: 0.6475986883835219
test_r2: 0.370961338035103
feature number: 15
train_rmse: 0.5585982343562808
test_rmse: 0.6532877281477496
test_r2: 0.35977360722279716
feature number: 16
train_rmse: 0.5583046173338214
test_rmse: 0.6545665452340732
test_r2: 0.3572135415140756
feature number: 17
train_rmse: 0.5576729798800557
test_rmse: 0.6571029932149088
test_r2: 0.35139887537569386
feature number: 18
train_rmse: 0.5571615503294346
test_rmse: 0.6586201157845674
test_r2: 0.3481844135697164
feature number: 19
train_rmse: 0.5568861479146975
test_rmse: 0.6604682587467925
test_r2: 0.34465925371832284
feature number: 20
train_rmse: 0.5565213290456121
test_rmse: 0.6614752434492438
test_r2: 0.3427251110218976
Test_rmse_min = 1.fea_num: 13  2.rmse: 0.6461008322537474
Test_r2_max = 1. fea_num: 13  2.r2: 0.37433818993363543


In [15]:
#
def build_model():
	model = models.Sequential()
#
	model.add(layers.Dense(4, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
	model.add(layers.Dense(1, activation='linear'))
	model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

	return model

In [18]:
fea_num=[5,10,15] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('../BayesianRidge_Pre_0/Input_A1_020.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')
    

feature number: 5
test_rmse_ave: 0.6331242525889584
[0.5350122707821986, 0.6659577887539428, 0.8275469860793463, 0.667227800349955, 0.6550376267788615, 0.757453560409197, 0.7505268025909392, 0.612149760170135, 0.4303998062156342, 0.4299301237593736]


test_r2_ave: 0.31327615101370987
[0.46283810153941485, 0.3537509532429888, 0.4033075278977917, -0.35336788264292807, 0.5111226950237278, 0.21127590718090905, 0.23136586665237568, 0.33243649709625367, 0.5578993174696685, 0.4221325266768967]


feature number: 10
test_rmse_ave: 0.6106311017769028
[0.4790951349857293, 0.7027290647675556, 0.8274987068649231, 0.5362569200758793, 0.6059662624936278, 0.7317327523249888, 0.7158951289103913, 0.6033115447607469, 0.4334708921553742, 0.4703546104298117]


test_r2_ave: 0.3736558817349852
[0.5692540171954044, 0.2804146005743835, 0.4033771481222935, 0.12579473297939847, 0.5816264000367484, 0.2639317695037865, 0.3006637041052954, 0.35157389721384646, 0.5515676548693207, 0.3083548927493741]


feature numbe